All imports:

In [1]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math

Define display function

In [2]:
def display_image(mat_like, cmap="rgb"):
    plt.figure()
    if cmap != "rgb":
        plt.imshow(mat_like, cmap=cmap)
    else:
        plt.imshow(mat_like)
    plt.show()
    plt.close()

load root folder and file location

In [3]:
current_folder = os.getcwd()
root_data_folder = os.path.join(current_folder, "local_data")
random_frames_folder = os.path.join(root_data_folder, "random_frames")
sequence_1_folder = os.path.join(root_data_folder, "sequence_1")
sequence_2_folder = os.path.join(root_data_folder, "sequence_2")
sequence_3_folder = os.path.join(root_data_folder, "sequence_3")
sequence_4_folder = os.path.join(root_data_folder, "sequence_4")

Clean image function

In [4]:
dil_kernel_size = 15
dil_kernel = cv2.getStructuringElement(
    cv2.MORPH_ELLIPSE, (dil_kernel_size, dil_kernel_size)
)


def clean_image(gray, display=False):
    gray = cv2.medianBlur(gray, 31)  # blur
    if display:
        display_image(gray, "gray")

    gray = cv2.equalizeHist(gray)  # equalize
    if display:
        display_image(gray, "gray")

    gray = 255 - gray  # inverse
    if display:
        display_image(gray, "gray")

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(13, 13))
    gray = clahe.apply(gray)  # clahe
    if display:
        display_image(gray, "gray")

    _, gray = cv2.threshold(gray, 230, 255, cv2.THRESH_BINARY)  # threshold
    if display:
        display_image(gray, "gray")

    gray = cv2.dilate(gray, dil_kernel, iterations=3)  # dilate
    if display:
        display_image(gray, "gray")

    gray = cv2.erode(gray, dil_kernel, iterations=8)  # erode
    if display:
        display_image(gray, "gray")

    gray = cv2.dilate(gray, dil_kernel, iterations=7)
    if display:
        display_image(gray, "gray")

    return gray

find circles function

In [5]:
def get_circles_from_gray(gray, color=None):
    gray = clean_image(gray)
    # display_image(gray, "gray")
    circles = cv2.HoughCircles(
        gray,
        cv2.HOUGH_GRADIENT,
        dp=1,  # down sample size
        minDist=500,  # minimum distance between detected circles
        param1=10,
        param2=14,
        minRadius=50,
        maxRadius=100,
    )

    if circles is None:
        print("no matches")
        return []

    color_img_cpy = np.copy(color)
    circles = np.uint(np.around(circles))
    circles = circles[0, :]

    if color is None:
        return circles

    for circle in circles:
        x, y, r = circle[0], circle[1], circle[2]
        cv2.circle(color_img_cpy, (x, y), r, (255, 255, 255), 15)
    # display_image(color_img_cpy)
    return circles


def merge_circle_arr(res, add):
    for circle_n in add:
        x_new, y_new, r_new = circle_n[0], circle_n[1], circle_n[2]
        for circle_o in res:
            x_old, y_old, r_old = circle_o[0], circle_o[1], circle_o[2]
            if math.dist((x_new, y_new), (x_old, y_old)) < 50:
                x_new = (x_new + x_old) / 2
                y_new = (y_new + y_old) / 2
                r_new = max(r_new, r_old)
                circle_o[0], circle_o[1], circle_o[2] = x_new, y_new, r_new
                r_new = -1
                break

        if r_new > 0:
            res.append(circle_n)


def find_colored_circles(color, display=False):
    r, g, b = cv2.split(color)

    r_circles = get_circles_from_gray(r, color)
    g_circles = get_circles_from_gray(g, color)
    b_circles = get_circles_from_gray(b, color)

    result = []
    merge_circle_arr(result, r_circles)
    merge_circle_arr(result, g_circles)
    merge_circle_arr(result, b_circles)

    if not display:
        return result

    img_copy = np.copy(color)
    for circle in result:
        x, y, r = circle[0], circle[1], circle[2]
        cv2.circle(img_copy, (x, y), r, (255, 255, 255), 15)

    display_image(img_copy)
    return result

load one image for testing

In [8]:
def explore_one_image(img_path):
    rnd_img = cv2.imread(img_path)
    color_rgb = cv2.cvtColor(rnd_img, cv2.COLOR_BGR2RGB)
    circles = find_colored_circles(color_rgb)
    
    result = []
    for circle in circles:
        x, y, r = circle[0], circle[1], circle[2]
        mask = np.zeros(color_rgb.shape[:2], dtype=np.uint8)
        cv2.circle(mask, (x, y), r // 2, 255, thickness=-1)
        mean_color = cv2.mean(color_rgb, mask=mask)
        mean_color = np.uint(np.around(mean_color))[:3]
        result.append([x, y, mean_color])
        
    return result

img_path = os.path.join(sequence_4_folder, "seq_000.jpg")

explore_one_image(img_path)

[[np.uint64(1328), np.uint64(993), array([ 90, 110,  74], dtype=uint64)],
 [np.uint64(444), np.uint64(868), array([64, 76, 77], dtype=uint64)],
 [np.uint64(1184), np.uint64(109), array([66, 76, 78], dtype=uint64)],
 [np.uint64(828), np.uint64(483), array([ 70,  94, 121], dtype=uint64)],
 [np.uint64(1718), np.uint64(612), array([117,  78,  60], dtype=uint64)]]

Now starting part 2 of the exercise!

In [ ]:
def analyze_sequence(folder, file_name_start):
    result = pd.DataFrame()
    result[""]